# Generative Learning Algorithms

- Algorithms that learn $p(y|x)$ directly from the training data are called the **descriminative learning algorithms**.
- Algorithms that model $p(x|y)$ and $p(y)$ from the training examples are called the **generative models**.
e.g. For a binary classification problem, we can learn $p(x|y=0)$ and $p(x|y=1)$  i.e. given the class labels $y$ how does the distribution of features look like. We'll also model the class distribution $p(y)$ and find the posterior distribution $p(y|x)$ for any new training example $x$ - 
$$
p(y|x) = \frac{p(x|y)p(y)}{p(x|y=0)p(y=0) + p(x|y=1)p(y=1)}
$$ 
To make the predictions - 
$$\begin{align*}
\hat{y} &= argmax_yp(y|x) = argmax_y\frac{p(x|y)p(y)}{p(x)} \\
&= argmax_yp(x|y)p(y)
\end{align*}
$$

## Gaussian Discriminant Analysis (GDA)
When $p(x|y)$ is distributed as multivariate Gaussian Distribution. 

### Multivariate Normal Distribution - 
Probability distribution function is given by - 
$$
p(x; \mu, \Sigma) = \frac{1}{(2\pi)^{d/2}|\Sigma|^{1/2}}exp(-\frac{1}{2}(x - \mu)^T\Sigma^{-1}(x - \mu))
$$
Where -
- $\mu \in \bf{R}^d$ - mean vector 
- $\Sigma \in \bf{R}^{d \times d}$ - covariance matrix, symmetric and positive definite
- $|\Sigma|$ - determinant of the covariance matrix